# Robustness Testing

---

## Overview
K-means is initiated using a random process, and therefore can give different answers based on the random initiation. It is important we make sure our k-means setup is robust, and returns very similar results every time. K-means not returning thsimilar results can be indicative of a few things. It can often be a symptom of the value of k being too small, or it can mean that n_init is not high enough. The n_init variable is how many times k-means clustering is preformed for each call of the k-means function. The initiation with the best final result (most compact clusters) will be chosen as the final output. The larger n_init is the longer the computation time but the more robust the final output is. 
i
In previous works ([Tselioudis et al. 2013](https://journals.ametsoc.org/view/journals/clim/26/19/jcli-d-13-00024.1.xml)) have tested for a robust k-means setup by preforming the clustering three times for the final k and checking that the correlations between the cluster centers is > 0.8.

This script will preform kmeans clustering n_trials times, and test if correlations between the cluster centers is > correlation_limit, where n_trials and correlation_limit are user defined variables. Feel free to start with n_trials=3 and correlation_limit=0.8, but if computation power allows it may be worth experimenting with more rigourous requirements. 

---

## Imports

In [ ]:
import numpy as np
from Functions import emd_means, euclidean_kmeans, plot_hists, open_and_process, plot_rfo
import logging as lgr
import dask
import xarray as xr

## Defining Variables and Opening Data

Here we define the variables necesary to begin using the toy ISCCP dataset included with this cookbook. To start off leave these variables alone, but later on feel free to experiment and change n_trials and correlation_limit. Also free to mess around with the k-means properties. Try increasing tol or decreasing n_init, try an innapropriate value of k or random initiation vs k-means++. Change plot_cr_centers or plot_rfo_graphs to true to plot the clustering results from each trial and examine how similar they are. It is also worth noting that just because a k-means setup is robust for the relatively small ISCCP dataset we're using, does not mean it will be robust for much larger datasets.

If running locally with your own dataset, you will need to change these to match your data.

In [ ]:
# Path to data to cluster
data_path = "./ISCCP_toy_data.nc"

# Path to the directory you wish to save plots in if running as a script, if None plots will only be shown and not saved. Enter as String
save_path = None
save_prefix = None  # prefix to put into the name of each plot to distinguish different runs of this script

# Variable name of data to cluster in data_path
# Name of tau dimension for var_name
# Name of height/pressure dimension for var_name
var_name =  'n_pctaudist' 
tau_var_name =  'levtau' 
ht_var_name =  'levpc'
lat_var_name = 'lat'
lon_var_name = 'lon'

# Does this dataset use cloud top height or cloud top pressure? enter "h" for height or "p" for pressure
height_or_pressure = 'p'

# kmeans properties
k = 4   # number of cluster to create
tol = 0.001   # maximum change in inertia values between kmeans iterations to declare convergence. should be higher if using wasserstein distance
max_iter = 100   # maximum number of k-means iterations to preform for each initiation
init = 'k-means++'    # initialization technique for kmeans, can be 'k-means++', 'random', or initial clusters to use of shape (k, n_tau_bins * n_pressure_bins)
n_init = 10    # number of initiations of the k-means algorithm. The final result will be the initiation with the lowest calculated inertia
gpu = False  # If the user has an Nvidia GPU, euclidean clustering can br preformed on it for a very significant speed up. CUPY/CUML must be installed in conda environment.

# k sensitivity testing properties
n_trials = 3 # how many times to preform the clustering with above properties
correlation_limit = 0.8 # Lowest acceptable correlation between the CRs through n_trials

# Want to see the results of clustering for each trial? Set one or both of these to true. 
plot_cr_centers = False
plot_rfo_graphs = False

# Choose whether to use a euclidean or wasserstein distance kmeans algorithm
wasserstein_or_euclidean = "euclidean"

# Minimum and Maximum longitudes and latitudes entered as list, or None for entire range: Ex [-65,65]
lat_range = [-90,90]
lon_range = [-180,180]

# Time Range min and max, or None for all time, entered as list of str: Ex. ["2003-03-01", "2004-07-01"] or ['2003','2007']
time_range = None

# Use data only over land or over ocean
# Set to 'L' for land only, 'O' for ocean only, or False for both land and ocean
only_ocean_or_land = False
# Does this dataset have a built in variable for land fraction? if so enter variable name as a string, otherwise cartopy will be used to mask out land or water
land_frac_var_name = None

## Opening and Proprocessing Data

In [ ]:
# Logging level, set to "INFO" for information about what the code is doing, otherwise keep at "WARNING"
logging_level = 'INFO'

# Setting up logger
lgr.basicConfig(level=lgr.DEBUG)
# Concatenating save_path and save prefix
if save_path != None: save_path = save_path + save_prefix
# Avoid creation of large chunks with dask
dask.config.set({"array.slicing.split_large_chunks": False})
# Automatically setting premade_cloud_regimes to none because this file does not need them. Do not Change.
premade_cloud_regimes = None

# Opening and preprocessing data
mat, valid_indicies, ds, histograms, weights = open_and_process(data_path, k, tol, max_iter, init, n_init, var_name, tau_var_name, ht_var_name, lat_var_name, lon_var_name, height_or_pressure, wasserstein_or_euclidean, premade_cloud_regimes, lat_range, lon_range, time_range, only_ocean_or_land, land_frac_var_name, cluster = False)

# Setting up array to hold reults of every clustering trial
cl_trials = np.empty((n_trials,k,mat.shape[1]))

## Clustering and Checking Results

In [ ]:
# Preform clustering with specified distance metric, n_trials times and record the resultant cloud regimes each time
for trial in range(n_trials):
    if wasserstein_or_euclidean == "wasserstein":
        cl_trials[trial], cluster_labels_temp, throw_away, throw_away2 = emd_means(mat, k, tol, init, n_init, ds, tau_var_name, ht_var_name, hard_stop = 45, weights = None)
    elif wasserstein_or_euclidean == "euclidean":
        cl_trials[trial], cluster_labels_temp = euclidean_kmeans(k, init, n_init, mat, max_iter, tol, gpu)
    else: raise Exception ('Invalid option for wasserstein_or_euclidean. Please enter "wasserstein", "euclidean"')

    if plot_cr_centers or plot_rfo_graphs:
        # Reshaping cluster_labels_temp to original shape of ds and reinserting NaNs in the original places, so plot can be made if needed
        cluster_labels = np.full(len(histograms), np.nan, dtype=np.int32)
        cluster_labels[valid_indicies]=cluster_labels_temp
        cluster_labels = xr.DataArray(data=cluster_labels, coords={"spacetime":histograms.spacetime},dims=("spacetime") )
        cluster_labels = cluster_labels.unstack()

        if plot_cr_centers:
            plot_hists(cluster_labels, k, ds, ht_var_name, tau_var_name, valid_indicies, mat, cluster_labels_temp, height_or_pressure, save_path)

        if plot_rfo_graphs:
            plot_rfo(cluster_labels,k,ds, save_path)

# Testing the correlation coefficients of resultant CRs across k-means runs
cor_coefs = np.zeros((k,k))
failures = 0
for trial1 in range(n_trials):
    cl1 = cl_trials[trial1]
    for trial2 in range(n_trials):
        cl2 = cl_trials[trial2]
        for i in range (k):
            for x in range (k):
                cor_coefs[i,x] = np.corrcoef(cl1[i].flatten(),cl2[x].flatten())[0,1]
        max_cors = np.max(cor_coefs, axis=1)
        if np.min(max_cors) < correlation_limit: failures += 1
if failures > 0:
    print(f"Failed robustness testing. {failures} centroids have a correlation of less than {correlation_limit} with the other k-means initiations")
else:
    print('Success: This K-means setup was insensitive to the initial centroids used')

Once you're happy with the results and are confident the k-means setup is robust, run the next cell to save the clustering results from the last trial, and open the next file plot_and_analyze.ipynb.

In [ ]:
np.save("./toy_ISCCP_cluster_centers.npy", cl_trials[-1])